In [215]:
import numpy as np
import pandas as pd
import igraph
import random as rnd
import csv

### 必要参数

In [216]:
num_nodes = 2000
p_rewire = .01
z = 12
a = 2
mod = 3
end_ratio = .99 # stop the contagion if proportion is greater than this ratio
echo = True # whether print the every steps

### 生成网络

In [217]:
# 1. create the network

# create a small world without randomness
g = igraph.Graph.Watts_Strogatz(dim = 1,
    size = num_nodes,
    nei = int(z/2),
    p = 0)
# Randomly rewires the graph while preserving the degree distribution
g.rewire(n = int(g.ecount() * 0.5 * p_rewire),
    mode = "simple")

### 初始化状态

In [218]:
# 2. set up states

# the initial node and its neighbors are activited
focal_node = rnd.sample(range(num_nodes), 1)[0]
initial_activate = g.neighborhood(vertices=focal_node,
    order = 1)
State = np.zeros(shape = (num_nodes,))
State[initial_activate, ] = 1

In [219]:
State.shape

(2000,)

### 确定权重矩阵

In [220]:
# 3. weight matrix
adj_matrix = g.get_adjacency_sparse() # get adjacency matrix


# mode1
if mod == 1:
    WM = adj_matrix[:,:]
# mode2 and mode3
else:
    # calculate edge betweeness
    Centrality = np.array(g.edge_betweenness())
    cut_points = np.percentile(Centrality, q=[25, 75])

    # change the value based on cut_points
    Centrality[Centrality < cut_points[0]] = 1  # ties embedded in small circle
    Centrality[(Centrality >= cut_points[0]) & (Centrality < cut_points[1])] = 2
    Centrality[Centrality >= cut_points[1]] = 3 # long-range ties
    
    # mode2
    if mod == 2:
        WM = adj_matrix[:,:]
        # get index of strong ties
        id_strong_mod2 = np.where(Centrality == 1)[0]
        # change the matrix
        for source_target in g.es[list(id_strong_mod2)]:
            i, j = source_target.tuple
            WM[i, j] = 5
            WM[j, i] = 5
    
    # mode3
    else:
        WM = adj_matrix[:,:]
        # get index of strong ties
        id_strong_embed = np.random.choice(np.where(Centrality == 1)[0], 
            int(g.ecount() * .2), replace=False)
        id_strong_long_range = np.random.choice(np.where(Centrality == 3)[0], 
            int(g.ecount() * .05), replace=False)
        id_strong_mod3 = np.hstack((id_strong_embed, id_strong_long_range))
        # chang the matrix
        for source_target in g.es[list(id_strong_mod3)]:
            i, j = source_target.tuple
            WM[i, j] = 5
            WM[j, i] = 5

### 进行传播

In [221]:
times = 0 # record steps
prop_path = [State.mean()] # record proportions

# contagion process
while prop_path[-1] <= end_ratio:

    # nonactivated nodes index:
    nonactivated_index = np.where(State != 1)[0]

    # update these nonactivated nodes in random order
    np.random.shuffle(nonactivated_index)
    for i in nonactivated_index:
        total_effects = np.dot(State, WM[:, i].toarray())[0]
        if total_effects >= a:
            State[i] = 1
    
    # recording after one-step iteration
    times += 1
    prop_path.append(State.mean())

    # print this step 
    if echo:
        print('第{}次迭代后:  {}的节点被激活'.format(times, prop_path[-1]))
    
    # stop the contagion if necessary
    if prop_path[-1] == prop_path[-2]:
        times[-1] = 9999
        break

else:
    # write csv
    with open("sim_record.csv", "a") as myfile:
        writer = csv.writer(myfile)
        writer.writerow([mod, num_nodes, z, a, p_rewire, times])
    print('**模式{}:N={},z={}, a={}, p_rewire={}, 时长{}'.format(mod, num_nodes, z, a, p_rewire, times))

第1次迭代后:  0.0235的节点被激活
第2次迭代后:  0.0575的节点被激活
第3次迭代后:  0.077的节点被激活
第4次迭代后:  0.1055的节点被激活
第5次迭代后:  0.144的节点被激活
第6次迭代后:  0.209的节点被激活
第7次迭代后:  0.2875的节点被激活
第8次迭代后:  0.364的节点被激活
第9次迭代后:  0.436的节点被激活
第10次迭代后:  0.5175的节点被激活
第11次迭代后:  0.605的节点被激活
第12次迭代后:  0.7的节点被激活
第13次迭代后:  0.8245的节点被激活
第14次迭代后:  0.9085的节点被激活
第15次迭代后:  0.9595的节点被激活
第16次迭代后:  0.9905的节点被激活
**模式3:N=2000,z=12, a=2, p_rewire=0.01, 时长16
